<div class="alert alert-block alert-info text-center">
    <H1> PHASE I (cont.) - preparing data </H1>
</div>


## Getting all dataframes ready to explore

- Data In & Data Out
- Getting the price history for each ticker
- Getting business information to enrich the analysis
- Understanding the big picture

In [ ]:
import pandas as pd
from glob import glob
from time import strftime, sleep
import numpy as np
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay
import yfinance as yf
yf.pdr_override()
import BizExtractor

def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '', regex=False).str.replace('(', \
                '', regex=False).str.replace(')', '', regex=False).str.replace(' ', '_', regex=False).str.replace('_/_', '/', regex=False)
    
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

def datetime_maker(df, datecol):
    df[datecol] = pd.to_datetime(df[datecol])

#### Import the last transactions_finaldf from Phase I

In [ ]:
last_file = sorted(glob('../outputs/transactions_all/*finaldf*.xlsx'))[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
all_transactions = pd.read_excel(last_file, engine='openpyxl')
all_transactions.date = pd.to_datetime(all_transactions.date, format='%d/%m/%Y')

In [ ]:
all_tickers = list(all_transactions['ticker'].unique())
print('You have {} different stocks'.format(len(all_tickers)))

In [ ]:
all_tickers

In [ ]:
# All transactions without the delisted stocks
# final_filtered = all_transactions[~all_transactions.ticker.isin(blacklist)]

### Collecting the price history for all tickers

- You can define the start date for the history below
- Datareader will get each stock individually
- all_data will have all the prices for every ticker

In [ ]:
ly = datetime.today().year-1
today = datetime.today()
start_sp = datetime(2020, 1, 1)
end_sp = today
start_stocks = datetime(2020, 1, 1)
end_stocks = today
start_ytd = datetime(ly, 12, 31) + BDay(1) # to get the first business day

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))
               
all_data = get(all_tickers, start_stocks, end_stocks)

In [ ]:
all_data

In [ ]:
clean_header(all_data)

In [ ]:
all_data.loc['TSLA']

### Saving the price history

In [ ]:

blacklist = []
for tick in all_tickers:
    try:
        all_data.loc[tick].to_csv('../outputs/price_hist/{}_price_hist.csv'.format(tick))
    except KeyError:
        blacklist.append(tick)
        print(f'Ticker {tick} has no price history to save.')
        pass
blacklist

In [ ]:
filt_tickers = [tick for tick in all_tickers if tick not in blacklist]

In [ ]:
all_data.info()

In [ ]:
MEGA_DICT = {}  
min_date = '2020-01-01'  # optional
TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
tx_filt = all_transactions[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in filt_tickers:
    prices_df = all_data[all_data.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'adj_close']
    prices_df = prices_df[prices_df.date >= min_date][PX_COLS].set_index(['date'])
    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('date').agg({'cashflow': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna('-')
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace='-', method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace='-', method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace='-', method='ffill')
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace='-', value=0)
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace='-', value=0)
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace='-', value=0)
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].replace(to_replace='-', value=0)
    # Cumulative sum for the cashflow
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].cumsum()
    tx_and_prices[['cml_cost', 'cml_units']] = tx_and_prices[['cml_cost', 'cml_units']].apply(pd.to_numeric)
    tx_and_prices['avg_price'] = round(tx_and_prices['cml_cost']/tx_and_prices['cml_units'],3)
    tx_and_prices['mktvalue'] = round(tx_and_prices['cml_units']*tx_and_prices['adj_close'],3)
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices

In [ ]:
MEGA_DICT['AAPL'].head(10)

In [ ]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)
MEGA_DF.to_csv('../outputs/mega/MEGA_DF_{}.csv'.format(get_now()))
MEGA_DF.info()

In [ ]:
last_file = sorted(glob('../outputs/mega/MEGA*.csv'))[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
MEGA_DF = pd.read_csv(last_file)

MEGA_DF['date'] = pd.to_datetime(MEGA_DF['date'])
MEGA_DF.set_index('date', inplace=True)

## Creating the daily snapshots of our portfolio

- Taking the MEGA_DF dataframe, we keep just the "market value" column for each stock
- If we sum the rows, it will show us how much our portfolio was worth on that day
- Adding SP500 for reference and calculating some metrics
- saving the portf_allvalues dataframe as csv

In [ ]:
portf_allvalues = MEGA_DF.filter(regex='mktvalue').fillna(0)
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1)
portf_allvalues['portf_value']

In [ ]:
# For the S&P500 price return
# You can use other symbols. Look it up on yahoo finance
sp500 = pdr.get_data_yahoo('^GSPC', start_stocks, end_sp)
clean_header(sp500)

In [ ]:
#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['adj_close'], how='inner')
portf_allvalues.rename(columns={'adj_close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)
portf_allvalues.index.name = 'date'
portf_allvalues.reset_index(inplace=True)
portf_allvalues

In [ ]:
portf_allvalues.to_csv('../outputs/portfolio_df/portfolio_df_{}.csv'.format(get_now()), index=False)

## Getting Sector and Industry
### The "give me everything" method

- In order for us to get a broader view on our portfolio, we want to add the sector and industry to the dataframe
- If you want a complete view of each company, go ahead and use the GET_BIZ_DATA_ALL function
- It takes a bit longer if you have a lot of tickers on your list, but it gives you a lot of information to explore later

In [ ]:
%%time
screener_all = BizExtractor.get_biz_data_all(all_tickers)

In [ ]:
# Visit https://finviz.com/screener.ashx

screener_all.info()

In [ ]:
screener_all.to_csv('../outputs/ticker_information/ticker_information.csv', index=False)

In [ ]:
cols_clean_all = ['Ticker', 'Company', 'Sector', 'Industry', 'P/E', 'Perf Week', 'Perf Month', 'Perf Quart',
             'Perf Half', 'Perf Year', 'Perf YTD', 'Volatility W', 'Volatility M', 'Recom', 'ATR',
             'SMA20', 'SMA50', 'SMA200', '52W High', '52W Low', 'RSI', 'Insider Own', 'Insider Trans',
             'Inst Own', 'Inst Trans', 'Float Short', 'Short Ratio', 'Dividend', 'LTDebt/Eq', 'Debt/Eq']
screener_all_clean = screener_all[cols_clean_all]
clean_header(screener_all_clean)
screener_all_clean.info()

In [ ]:
%%time
finviz_merged_raw = BizExtractor.get_finviz_screener('OVERVIEW', all_tickers)

In [ ]:
cols_clean = ['Ticker', 'Company', 'Sector', 'Industry', 'P/E']
finviz_merged_clean = finviz_merged_raw[cols_clean]
clean_header(finviz_merged_clean)
finviz_merged_clean.info()

# Last Positions

## Getting the latest values

- In order to get the latest position value, we need to get the latest prices from yahoo finance
- Since we also want sector and industry to be able to segment our portfolio, we will get data from finviz too


In [ ]:
last_positions = all_transactions.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'cashflow': 'sum'}).reset_index()
last_positions.head(10)

In [ ]:
%%time
curr_prices = []
for tick in last_positions['ticker']:
    try:
        price = yf.download(tick, interval='1mo', period='d')['Adj Close'][-1]
    except:
        print(f'No price info for {tick}')
        price = 0
    curr_prices.append(price)
len(curr_prices)

In [ ]:
last_positions['price'] = curr_prices
last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)
last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)
last_positions = last_positions.sort_values(by='current_value', ascending=False)

last_positions.tail(10)

## Final Last Positions

In [ ]:
# final_lastpositions = pd.merge(finviz_merged_clean, last_positions, left_on='ticker', right_on='ticker', how='outer')
final_lastpositions = pd.merge(screener_all_clean, last_positions, left_on='ticker', right_on='ticker', how='outer')
final_lastpositions['current_value'] = final_lastpositions.price*final_lastpositions.cml_units
final_lastpositions['avg_price'] = final_lastpositions.cml_cost/final_lastpositions.cml_units
final_lastpositions['portf_weight'] = final_lastpositions.current_value/final_lastpositions.current_value.sum()*100
final_lastpositions['unrealizedval'] = final_lastpositions.current_value - final_lastpositions.cml_cost
final_lastpositions['unrealizedpct'] = final_lastpositions.unrealizedval / final_lastpositions.cml_cost*100
final_lastpositions.round(2)

In [ ]:
# final cosmetics
final_lastpositions = final_lastpositions.replace('-', '0')
for c in final_lastpositions.iloc[:,4:30].columns:
    final_lastpositions[c] = pd.to_numeric(final_lastpositions[c].str.replace('%', ''))
final_lastpositions.info()

In [ ]:
final_lastpositions.round(2).to_csv('../outputs/final_current_positions/final_current_positions_{}.csv'.format(get_now()), index=False)

## Grouped DF's

In [ ]:
grouped_sect = final_lastpositions.groupby(['sector']).agg(
    {'ticker': 'count', 'current_value': 'sum', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by='current_value', ascending= False).reset_index().round(2)
grouped_sect['weight'] = round(grouped_sect.current_value/grouped_sect.current_value.sum()*100, 2)
grouped_sect

In [ ]:
grouped_sect.to_csv('../outputs/grouped/grouped_sect_{}.csv'.format(get_now()), index=False)

In [ ]:
grouped_sect_ind = final_lastpositions.groupby(['sector', 'industry']).agg(
    {'ticker': 'count', 'current_value': 'sum', 'cml_cost': 'sum', 'gain_loss': 'sum'}
).sort_values(by='current_value', ascending= False).reset_index().round(2)
grouped_sect_ind['weight'] = round(grouped_sect_ind.current_value/grouped_sect_ind.current_value.sum()*100, 2)
grouped_sect_ind

In [ ]:
grouped_sect_ind.to_csv('../outputs/grouped/grouped_sect_ind_{}.csv'.format(get_now()), index=False)